<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/PP/move_closer_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install osmnx momepy geopandas > /dev/null

ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.26.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [3]:
from pathlib import Path
import os
from glob import glob

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.optimize import minimize

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

from lightgbm import LGBMModel

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import plotly.express as px

from shapely.geometry import Point
import osmnx as ox
import momepy
import geopandas as gpd

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:110: UserWarning:

The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.



In [6]:
path = Path("/content/drive/MyDrive/GSDC")
train_base = pd.read_csv(path / "baseline_locations_train.csv")
test_base = pd.read_csv(path / "baseline_locations_test.csv")

#train_base = pd.read_csv(path / "baseline_locations_train.csv")
#test_base = pd.read_csv("/content/drive/MyDrive/GSDC/test_predict_next_point_SJC_2.csv")

print(len(train_base["collectionName"].unique()))
print(len(train_base["phone"].unique()))

29
73


In [8]:
def get_groundtruth(path: Path) -> pd.DataFrame:
        output_df = pd.DataFrame()
        
        for path in glob(str(path / 'train/*/*/ground_truth.csv')):
            _df = pd.read_csv(path)
            output_df = pd.concat([output_df, _df])
        output_df = output_df.reset_index(drop=True)
        
        _columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
        output_df[['t_'+col for col in _columns]] = output_df[_columns]
        output_df = output_df.drop(columns=_columns, axis=1)
        return output_df

train_base = train_base.merge(
    get_groundtruth(path),
    on=['collectionName', 'phoneName', 'millisSinceGpsEpoch']
)

In [9]:
train_SJC_list = ["2021-04-22-US-SJC-1", "2021-04-28-US-SJC-1", "2021-04-29-US-SJC-2"]
test_SJC_list = ["2021-04-02-US-SJC-1", "2021-04-22-US-SJC-2"]

train_SJCs = []
for SJC in train_SJC_list:
    SJC_df = train_base[train_base["collectionName"] == SJC]
    train_SJCs.append(SJC_df)
train_SJC = pd.concat(train_SJCs)

test_SJCs = []
for SJC in test_SJC_list:
    SJC_df = test_base[test_base["collectionName"] == SJC]
    test_SJCs.append(SJC_df)
test_SJC = pd.concat(test_SJCs)

In [10]:
def nearest_point(x, y, points):
    result = {}
    if len(points) == 0:
        return result
    result[0] = points[0][0]
    result[1] = points[0][1]
    stdval = math.sqrt((points[0][0] - x) ** 2 + (points[0][1] - y) ** 2)
    for point in points:
        distance = math.sqrt((point[0] - x) ** 2 + (point[1] - y) ** 2)
        if stdval > distance:
            result[0] = point[0]
            result[1] = point[1]
            stdval = distance
    return [result[0], result[1]]

    
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist

In [11]:
"""
道路の中心の座標を生成
各lat,lngから一番近い座標をmerge
"""

'\n道路の中心の座標を生成\n各lat,lngから一番近い座標をmerge\n'

In [12]:
def create_dataset(input_df):
    output_df = input_df.copy()
    collection_list = output_df["collectionName"].unique()

    nearest_point_dfs_list = []
    c = 1
    for collection in collection_list:
        print(f"now : {c}/{len(collection_list)}")
        c += 1
        print(collection)
        target_df = output_df[output_df["collectionName"] == collection].reset_index(drop=True)
        target_df["geometry"] = [Point(p) for p in target_df[["lngDeg", "latDeg"]].to_numpy()]
        target_gdf = gpd.GeoDataFrame(target_df, geometry=target_df["geometry"])

        offset = 0.1**5
        bbox = target_gdf.bounds + [-(offset+0.01), -(offset+0.01), offset+0.01, offset+0.01]
        east = bbox["minx"].min()
        west = bbox["maxx"].max()
        south = bbox["miny"].min()
        north = bbox["maxy"].max()
        G = ox.graph.graph_from_bbox(north, south, east, west, network_type='drive')

        nodes, edges = momepy.nx_to_gdf(G)
        
        edges = edges.dropna(subset=["geometry"]).reset_index(drop=True)
        hits = bbox.apply(lambda row: list(edges.sindex.intersection(row)), axis=1)
        tmp = pd.DataFrame({
            "pt_idx": np.repeat(hits.index, hits.apply(len)),
            "line_i": np.concatenate(hits.values)
        })
        tmp = tmp.join(edges.reset_index(drop=True), on="line_i")
        tmp = tmp.join(target_gdf.geometry.rename("point"), on="pt_idx")
        tmp = gpd.GeoDataFrame(tmp, geometry="geometry", crs=target_gdf.crs)

        tmp["snap_dist"] = tmp.geometry.distance(gpd.GeoSeries(tmp.point))

        tolerance = 0.0005  
        tmp = tmp.loc[tmp.snap_dist <= tolerance]
        tmp = tmp.sort_values(by=["snap_dist"])

        closest = tmp.groupby("pt_idx").first()
        closest = gpd.GeoDataFrame(closest, geometry="geometry")
        closest = closest.drop_duplicates("line_i").reset_index(drop=True)

        line_points_list = []
        split = 200  # param: number of split in each LineString
        for dist in range(0, split, 1):
            dist = dist/split
            line_points = closest["geometry"].interpolate(dist, normalized=True)
            line_points_list.append(line_points)
        line_points = pd.concat(line_points_list).reset_index(drop=True)
        line_points = line_points.reset_index().rename(columns={0:"geometry"})
        line_points["lngDeg"] = line_points["geometry"].x
        line_points["latDeg"] = line_points["geometry"].y

        

        
        line_points_ = line_points.loc[:, ["lngDeg", "latDeg"]]
        line_points_list = line_points_.values.tolist()

        nearest_point_list = []
        for lng, lat in zip(
            target_df["lngDeg"].to_numpy(),
            target_df["latDeg"].to_numpy()
        ):
            nearest_point_list.append(nearest_point(lng, lat, line_points_list))
        
        print(len(nearest_point_list))
        print(output_df[output_df["collectionName"] == collection].shape)

        nearest_point_dfs_list.append(pd.DataFrame(nearest_point_list))


    nearest_point_df = pd.concat(nearest_point_dfs_list)
    output_df["latDeg_center"] = nearest_point_df[1].values
    output_df["lngDeg_center"] = nearest_point_df[0].values

    output_df['meter'] = output_df.apply(
            lambda r: calc_haversine(
                r.latDeg, r.lngDeg, r.latDeg_center, r.lngDeg_center
            ),
            axis=1
        )
    
    return output_df

In [12]:
train_df = create_dataset(train_SJC)

now : 1/3
2021-04-22-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




5716
(5716, 15)
now : 2/3
2021-04-28-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4097
(4097, 15)
now : 3/3
2021-04-29-US-SJC-2


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4700
(4700, 15)


In [13]:
test_df = create_dataset(test_SJC)

now : 1/2
2021-04-02-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4749
(4749, 7)
now : 2/2
2021-04-22-US-SJC-2


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




2324
(2324, 7)


In [16]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    print("< from truth >")
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df

In [14]:
def check_score_center(input_df):
    output_df = input_df.copy()
    print("< from center >")
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.latDeg_center, r.lngDeg_center
        ),
        axis=1
    )
    
    meter_score = output_df['meter'].mean()
    print(f'error meter by center: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score center: {score}')
    
    return output_df


In [15]:
def check_score_center_truth(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    print("< from center_truth >")
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg_center, r.lngDeg_center, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df

In [17]:
for name, collection_df in train_df.groupby("collectionName"):
    phone_list = collection_df["phoneName"].unique()
    for phone in phone_list:
        print(f"collection:{name}    phone:{phone}")
        phone_df = collection_df[collection_df["phoneName"] == phone]
        from_truth = check_score(phone_df)
        from_center = check_score_center_truth(phone_df)
        print("----------------------------------------")
    print("----------------------------------------")

collection:2021-04-22-US-SJC-1    phone:Pixel4
< from truth >
error meter: 13.052250286321655
score: 24.65997800816365
< from center_truth >
error meter: 11.41882565090956
score: 22.88627366383489
----------------------------------------
collection:2021-04-22-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 10.708077810394476
score: 17.768175752284254
< from center_truth >
error meter: 8.055758010359764
score: 16.28326070553694
----------------------------------------
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:Pixel4
< from truth >
error meter: 8.566368686394854
score: 14.370939744170363
< from center_truth >
error meter: 8.43721334912065
score: 14.25823187855645
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 9.22580980649068
score: 20.170897642446484
< from center_truth >
error meter: 8.943343906612247
score: 19.745149455178222
------------------------------

In [ ]:
check_score(train_SJC)

< from truth >
error meter: 10.5084494053175
score: 19.432900281770625


,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,speedMps,courseDegree,t_latDeg,t_lngDeg,t_heightAboveWgs84EllipsoidM,meter
103937,2021-04-22-US-SJC-1,Pixel4,1303160575446,37.334582,-121.899408,-10.66,2021-04-22-US-SJC-1_Pixel4,325.45,1.8,0.0,0.0,330.5,37.334581,-121.899447,56.91,3.435414
103938,2021-04-22-US-SJC-1,Pixel4,1303160576446,37.334603,-121.899416,-6.76,2021-04-22-US-SJC-1_Pixel4,326.45,1.8,0.0,0.0,330.5,37.334581,-121.899447,56.91,3.639803
103939,2021-04-22-US-SJC-1,Pixel4,1303160577446,37.334579,-121.899418,-10.73,2021-04-22-US-SJC-1_Pixel4,327.45,1.8,0.0,0.0,330.5,37.334581,-121.899447,56.91,2.563412
103940,2021-04-22-US-SJC-1,Pixel4,1303160578446,37.334595,-121.899419,-6.35,2021-04-22-US-SJC-1_Pixel4,328.45,1.8,0.0,0.0,330.5,37.334581,-121.899447,56.91,2.903583
103941,2021-04-22-US-SJC-1,Pixel4,1303160579446,37.334590,-121.899406,-19.56,2021-04-22-US-SJC-1_Pixel4,329.45,1.8,0.0,0.0,330.5,37.334581,-121.899447,56.91,3.729471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131337,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760315000,37.334460,-121.899600,-8.09,2021-04-29-US-SJC-2_SamsungS20Ultra,2632.00,1.1,0.0,0.0,60.0,37.334475,-121.899613,56.62,2.052491
131338,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760316000,37.334472,-121.899583,-7.59,2021-04-29-US-SJC-2_SamsungS20Ultra,2633.00,1.1,0.0,0.0,60.0,37.334475,-121.899613,56.62,2.671673
131339,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760317000,37.334491,-121.899597,-8.35,2021-04-29-US-SJC-2_SamsungS20Ultra,2634.00,0.9,0.0,0.0,60.0,37.334475,-121.899613,56.62,2.287458
131340,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760318000,37.334495,-121.899583,-8.73,2021-04-29-US-SJC-2_SamsungS20Ultra,2635.00,1.0,0.0,0.0,60.0,37.334475,-121.899613,56.63,3.454306


In [73]:
def Move_closer_train(input_df):
    input_df_ = input_df.copy()
    output_df = input_df.copy()
    collection_list = input_df_["collectionName"].unique()

    nearest_point_dfs_list = []
    c = 1
    for collection in collection_list:
        print(f"now : {c}/{len(collection_list)}")
        c += 1
        print(collection)
        target_df = input_df_[input_df_["collectionName"] == collection].reset_index(drop=True)
        target_df["geometry"] = [Point(p) for p in target_df[["lngDeg", "latDeg"]].to_numpy()]
        target_gdf = gpd.GeoDataFrame(target_df, geometry=target_df["geometry"])

        offset = 0.1**5
        bbox = target_gdf.bounds + [-(offset+0.01), -(offset+0.01), offset+0.01, offset+0.01]
        east = bbox["minx"].min()
        west = bbox["maxx"].max()
        south = bbox["miny"].min()
        north = bbox["maxy"].max()
        G = ox.graph.graph_from_bbox(north, south, east, west, network_type='drive')

        nodes, edges = momepy.nx_to_gdf(G)
        
        edges = edges.dropna(subset=["geometry"]).reset_index(drop=True)
        hits = bbox.apply(lambda row: list(edges.sindex.intersection(row)), axis=1)
        tmp = pd.DataFrame({
            "pt_idx": np.repeat(hits.index, hits.apply(len)),
            "line_i": np.concatenate(hits.values)
        })
        tmp = tmp.join(edges.reset_index(drop=True), on="line_i")
        tmp = tmp.join(target_gdf.geometry.rename("point"), on="pt_idx")
        tmp = gpd.GeoDataFrame(tmp, geometry="geometry", crs=target_gdf.crs)

        tmp["snap_dist"] = tmp.geometry.distance(gpd.GeoSeries(tmp.point))

        tolerance = 0.0005  
        tmp = tmp.loc[tmp.snap_dist <= tolerance]
        tmp = tmp.sort_values(by=["snap_dist"])

        closest = tmp.groupby("pt_idx").first()
        closest = gpd.GeoDataFrame(closest, geometry="geometry")
        closest = closest.drop_duplicates("line_i").reset_index(drop=True)

        line_points_list = []
        split = 200  # param: number of split in each LineString
        for dist in range(0, split, 1):
            dist = dist/split
            line_points = closest["geometry"].interpolate(dist, normalized=True)
            line_points_list.append(line_points)
        line_points = pd.concat(line_points_list).reset_index(drop=True)
        line_points = line_points.reset_index().rename(columns={0:"geometry"})
        line_points["lngDeg"] = line_points["geometry"].x
        line_points["latDeg"] = line_points["geometry"].y

        def nearest_point(x, y, points):
            result = {}
            if len(points) == 0:
                return result
            result[0] = points[0][0]
            result[1] = points[0][1]
            stdval = math.sqrt((points[0][0] - x) ** 2 + (points[0][1] - y) ** 2)
            for point in points:
                distance = math.sqrt((point[0] - x) ** 2 + (point[1] - y) ** 2)
                if stdval > distance:
                    result[0] = point[0]
                    result[1] = point[1]
                    stdval = distance
            return [result[0], result[1]]

        
        line_points_ = line_points.loc[:, ["lngDeg", "latDeg"]]
        line_points_list = line_points_.values.tolist()

        nearest_point_list = []
        for lng, lat in zip(
            target_df["lngDeg"].to_numpy(),
            target_df["latDeg"].to_numpy()
        ):
            nearest_point_list.append(nearest_point(lng, lat, line_points_list))
        
        print(len(nearest_point_list))
        print(input_df_[input_df_["collectionName"] == collection].shape)

        nearest_point_dfs_list.append(pd.DataFrame(nearest_point_list))

    nearest_point_df = pd.concat(nearest_point_dfs_list)
    input_df_["latDeg_center"] = nearest_point_df[1].values
    input_df_["lngDeg_center"] = nearest_point_df[0].values

    input_df_['meter'] = input_df_.apply(
            lambda r: calc_haversine(
                r.latDeg, r.lngDeg, r.latDeg_center, r.lngDeg_center
            ),
            axis=1
        )
    
    def check_meter(input_df):
        input_df_ = input_df.copy()
        lat_list = []
        lon_list = []
        for lat1, lon1, lat2, lon2, meter in zip(
            input_df_["latDeg"].to_numpy(),
            input_df_["lngDeg"].to_numpy(),
            input_df_["latDeg_center"].to_numpy(),
            input_df_["lngDeg_center"].to_numpy(),
            input_df_["meter"].to_numpy()
        ):
            if meter >= 6:
                lat1 = (lat1 + lat2)/2
                lon1 = (lon1 + lon2)/2
            elif meter >= 4:
                lat1 = ((lat1 + lat2)/2 + lat2)/2
                lon1 = ((lon1 + lon2)/2 + lon2)/2
            elif meter >= 2:
                lat1 = (((lat1 + lat2)/2 + lat2)/2 + lat2)/2
                lon1 = (((lon1 + lon2)/2 + lon2)/2 + lon2)/2
            lat_list.append(lat1)
            lon_list.append(lon1)

        output_df = pd.DataFrame()
        output_df["latDeg_fixed"] = lat_list
        output_df["lngDeg_fixed"] = lon_list
        return output_df

    check_meter_df = check_meter(input_df_)


    output_df["latDeg"] = check_meter_df["latDeg_fixed"].values
    output_df["lngDeg"] = check_meter_df["lngDeg_fixed"].values
    #output_df["t_latDeg"] = gt_df["latDeg"]
    #output_df["t_lngDeg"] = gt_df["lngDeg"]

    return output_df

In [55]:
train_SJC_move_closer = Move_closer_train(train_SJC)

now : 1/3
2021-04-22-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




5716
(5716, 15)
now : 2/3
2021-04-28-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4097
(4097, 15)
now : 3/3
2021-04-29-US-SJC-2


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4700
(4700, 15)


In [56]:
train_SJC_move_closer = Move_closer_train(train_SJC_move_closer)

now : 1/3
2021-04-22-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




5716
(5716, 15)
now : 2/3
2021-04-28-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4097
(4097, 15)
now : 3/3
2021-04-29-US-SJC-2


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4700
(4700, 15)


In [23]:
for name, collection_df in train_df.groupby("collectionName"):
    phone_list = collection_df["phoneName"].unique()
    for phone in phone_list:
        print(f"collection:{name}    phone:{phone}")
        phone_df = collection_df[collection_df["phoneName"] == phone]
        from_truth = check_score(phone_df)
        #from_center = check_score_center_truth(phone_df)
        print("----------------------------------------")
    print("----------------------------------------")

collection:2021-04-22-US-SJC-1    phone:Pixel4
< from truth >
error meter: 13.052250286321655
score: 24.65997800816365
----------------------------------------
collection:2021-04-22-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 10.708077810394476
score: 17.768175752284254
----------------------------------------
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:Pixel4
< from truth >
error meter: 8.566368686394854
score: 14.370939744170363
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 9.22580980649068
score: 20.170897642446484
----------------------------------------
----------------------------------------
collection:2021-04-29-US-SJC-2    phone:Pixel4
< from truth >
error meter: 10.815121991945263
score: 20.182595805584533
----------------------------------------
collection:2021-04-29-US-SJC-2    phone:SamsungS20Ultra
< from truth >
error meter: 9.6446550199061

In [57]:
#2回
for name, collection_df in train_SJC_move_closer.groupby("collectionName"):
    phone_list = collection_df["phoneName"].unique()
    for phone in phone_list:
        print(f"collection:{name}    phone:{phone}")
        phone_df = collection_df[collection_df["phoneName"] == phone]
        from_truth = check_score(phone_df)
        #from_center = check_score_center_truth(phone_df)
        print("----------------------------------------")
    print("----------------------------------------")

collection:2021-04-22-US-SJC-1    phone:Pixel4
< from truth >
error meter: 11.422274911616517
score: 22.75281006407258
----------------------------------------
collection:2021-04-22-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 7.98488925098054
score: 15.656478004663214
----------------------------------------
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:Pixel4
< from truth >
error meter: 8.145265612469641
score: 13.896069020092867
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 8.541571244157248
score: 19.078263641501906
----------------------------------------
----------------------------------------
collection:2021-04-29-US-SJC-2    phone:Pixel4
< from truth >
error meter: 9.42890391464834
score: 17.41542909672096
----------------------------------------
collection:2021-04-29-US-SJC-2    phone:SamsungS20Ultra
< from truth >
error meter: 8.320054162056756
s

In [29]:
"""
if meter >= 6:
                lat1 = (lat1 + lat2)/2
                lon1 = (lon1 + lon2)/2
            elif meter >= 4:
                lat1 = ((lat1 + lat2)/2 + lat2)/2
                lon1 = ((lon1 + lon2)/2 + lon2)/2
            elif meter >= 2:
                lat1 = (((lat1 + lat2)/2 + lat2)/2 + lat2)/2
                lon1 = (((lon1 + lon2)/2 + lon2)/2 + lon2)/2
"""
for name, collection_df in train_SJC_move_closer.groupby("collectionName"):
    phone_list = collection_df["phoneName"].unique()
    for phone in phone_list:
        print(f"collection:{name}    phone:{phone}")
        phone_df = collection_df[collection_df["phoneName"] == phone]
        from_truth = check_score(phone_df)
        #from_center = check_score_center_truth(phone_df)
        print("----------------------------------------")
    print("----------------------------------------")

collection:2021-04-22-US-SJC-1    phone:Pixel4
< from truth >
error meter: 11.751266041583039
score: 23.40716537634238
----------------------------------------
collection:2021-04-22-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 8.519576677909296
score: 15.393131541809145
----------------------------------------
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:Pixel4
< from truth >
error meter: 8.143414434075037
score: 13.297035040896867
----------------------------------------
collection:2021-04-28-US-SJC-1    phone:SamsungS20Ultra
< from truth >
error meter: 8.416046508088712
score: 18.6001240970395
----------------------------------------
----------------------------------------
collection:2021-04-29-US-SJC-2    phone:Pixel4
< from truth >
error meter: 9.646099615873615
score: 17.400944139627118
----------------------------------------
collection:2021-04-29-US-SJC-2    phone:SamsungS20Ultra
< from truth >
error meter: 8.235593737261864


In [14]:
def Move_closer(input_df):
    input_df_ = input_df.copy()
    output_df = input_df.copy()
    collection_list = input_df_["collectionName"].unique()

    nearest_point_dfs_list = []
    c = 1
    for collection in collection_list:
        print(f"now : {c}/{len(collection_list)}")
        c += 1
        print(collection)
        target_df = input_df_[input_df_["collectionName"] == collection].reset_index(drop=True)
        target_df["geometry"] = [Point(p) for p in target_df[["lngDeg", "latDeg"]].to_numpy()]
        target_gdf = gpd.GeoDataFrame(target_df, geometry=target_df["geometry"])

        offset = 0.1**5
        bbox = target_gdf.bounds + [-(offset+0.01), -(offset+0.01), offset+0.01, offset+0.01]
        east = bbox["minx"].min()
        west = bbox["maxx"].max()
        south = bbox["miny"].min()
        north = bbox["maxy"].max()
        G = ox.graph.graph_from_bbox(north, south, east, west, network_type='drive')

        nodes, edges = momepy.nx_to_gdf(G)
        
        edges = edges.dropna(subset=["geometry"]).reset_index(drop=True)
        hits = bbox.apply(lambda row: list(edges.sindex.intersection(row)), axis=1)
        tmp = pd.DataFrame({
            "pt_idx": np.repeat(hits.index, hits.apply(len)),
            "line_i": np.concatenate(hits.values)
        })
        tmp = tmp.join(edges.reset_index(drop=True), on="line_i")
        tmp = tmp.join(target_gdf.geometry.rename("point"), on="pt_idx")
        tmp = gpd.GeoDataFrame(tmp, geometry="geometry", crs=target_gdf.crs)

        tmp["snap_dist"] = tmp.geometry.distance(gpd.GeoSeries(tmp.point))

        tolerance = 0.0005  
        tmp = tmp.loc[tmp.snap_dist <= tolerance]
        tmp = tmp.sort_values(by=["snap_dist"])

        closest = tmp.groupby("pt_idx").first()
        closest = gpd.GeoDataFrame(closest, geometry="geometry")
        closest = closest.drop_duplicates("line_i").reset_index(drop=True)

        line_points_list = []
        split = 200  # param: number of split in each LineString
        for dist in range(0, split, 1):
            dist = dist/split
            line_points = closest["geometry"].interpolate(dist, normalized=True)
            line_points_list.append(line_points)
        line_points = pd.concat(line_points_list).reset_index(drop=True)
        line_points = line_points.reset_index().rename(columns={0:"geometry"})
        line_points["lngDeg"] = line_points["geometry"].x
        line_points["latDeg"] = line_points["geometry"].y

        def nearest_point(x, y, points):
            result = {}
            if len(points) == 0:
                return result
            result[0] = points[0][0]
            result[1] = points[0][1]
            stdval = math.sqrt((points[0][0] - x) ** 2 + (points[0][1] - y) ** 2)
            for point in points:
                distance = math.sqrt((point[0] - x) ** 2 + (point[1] - y) ** 2)
                if stdval > distance:
                    result[0] = point[0]
                    result[1] = point[1]
                    stdval = distance
            return [result[0], result[1]]

        
        line_points_ = line_points.loc[:, ["lngDeg", "latDeg"]]
        line_points_list = line_points_.values.tolist()

        nearest_point_list = []
        for lng, lat in zip(
            target_df["lngDeg"].to_numpy(),
            target_df["latDeg"].to_numpy()
        ):
            nearest_point_list.append(nearest_point(lng, lat, line_points_list))
        
        print(len(nearest_point_list))
        print(input_df_[input_df_["collectionName"] == collection].shape)

        nearest_point_dfs_list.append(pd.DataFrame(nearest_point_list))

    nearest_point_df = pd.concat(nearest_point_dfs_list)
    input_df_["latDeg_center"] = nearest_point_df[1].values
    input_df_["lngDeg_center"] = nearest_point_df[0].values

    input_df_['meter'] = input_df_.apply(
            lambda r: calc_haversine(
                r.latDeg, r.lngDeg, r.latDeg_center, r.lngDeg_center
            ),
            axis=1
        )
    
    def check_meter(input_df):
        input_df_ = input_df.copy()
        lat_list = []
        lon_list = []
        for lat1, lon1, lat2, lon2, meter in zip(
            input_df_["latDeg"].to_numpy(),
            input_df_["lngDeg"].to_numpy(),
            input_df_["latDeg_center"].to_numpy(),
            input_df_["lngDeg_center"].to_numpy(),
            input_df_["meter"].to_numpy()
        ):
            if meter >= 6:
                lat1 = (lat1 + lat2)/2
                lon1 = (lon1 + lon2)/2
            elif meter >= 4:
                lat1 = ((lat1 + lat2)/2 + lat2)/2
                lon1 = ((lon1 + lon2)/2 + lon2)/2
            elif meter >= 2:
                lat1 = (((lat1 + lat2)/2 + lat2)/2 + lat2)/2
                lon1 = (((lon1 + lon2)/2 + lon2)/2 + lon2)/2
            lat_list.append(lat1)
            lon_list.append(lon1)

        output_df = pd.DataFrame()
        output_df["latDeg_fixed"] = lat_list
        output_df["lngDeg_fixed"] = lon_list
        return output_df

    check_meter_df = check_meter(input_df_)


    output_df["latDeg"] = check_meter_df["latDeg_fixed"].values
    output_df["lngDeg"] = check_meter_df["lngDeg_fixed"].values
    

    return output_df

In [15]:
test = Move_closer(test_df)

now : 1/2
2021-04-02-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




4749
(4749, 10)
now : 2/2
2021-04-22-US-SJC-2


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




2324
(2324, 10)


In [16]:
test_base.iloc[62132:66881, 3] = test[test["collectionName"] == "2021-04-02-US-SJC-1"]["latDeg"].values
test_base.iloc[62132:66881, 4] = test[test["collectionName"] == "2021-04-02-US-SJC-1"]["lngDeg"].values

In [17]:
test_base.iloc[74067:76391, 3] = test[test["collectionName"] == "2021-04-22-US-SJC-2"]["latDeg"].values
test_base.iloc[74067:76391, 4] = test[test["collectionName"] == "2021-04-22-US-SJC-2"]["lngDeg"].values

In [21]:
test_base[sub.columns].to_csv("test_move_closer_SJC.csv", index=False)

In [89]:
test_base.to_csv("test_pre_next_move_closer_SJC.csv", index=False)

In [20]:
sub = pd.read_csv('/content/drive/MyDrive/GSDC/sample_submission.csv')

In [93]:
test_base[sub.columns].to_csv('test_pre_next_move_closer_SJC.csv', index=False)

In [94]:
a = pd.read_csv('/content/drive/MyDrive/GSDC/test_predict_next_point_SJC_2.csv')

In [95]:
a[sub.columns].to_csv('test_pre_next.csv', index=False)